<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/wikidata_sparql_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sparqlwrapper

# https://rdflib.github.io/sparqlwrapper/

     |████████████████████████████████| 348kB 8.0MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import re
from collections import Counter

def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
      item = []
      for c in cols:
        item.append(row.get(c, {}).get('value'))
      out.append(item)
    return pd.DataFrame(out, columns = cols)

In [4]:
pwd

'/content'

In [0]:
abv_dir = '/gdrive/My Drive/abv_reviews'


In [8]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [9]:
ls

 CoherenceScore.csv          pyLDAvis_10.html   pyLDAvis_25.html
 datalist_corrected.csv      pyLDAvis_11.html   pyLDAvis_26.html
 dictionary                  pyLDAvis_12.html   pyLDAvis_27.html
 doc_term_matrix.mm          pyLDAvis_13.html   pyLDAvis_28.html
 doc_term_matrix.mm.index    pyLDAvis_14.html   pyLDAvis_29.html
 images/                     pyLDAvis_15.html   pyLDAvis_3.html
 lda                         pyLDAvis_16.html   pyLDAvis_4.html
 lda.expElogbeta.npy         pyLDAvis_17.html   pyLDAvis_5.html
 lda_final                   pyLDAvis_18.html   pyLDAvis_6.html
 lda_final.expElogbeta.npy   pyLDAvis_19.html   pyLDAvis_7.html
 lda_final.id2word           pyLDAvis_20.html   pyLDAvis_8.html
 lda_final.state             pyLDAvis_21.html   pyLDAvis_9.html
 lda.id2word                 pyLDAvis_22.html   reviews_text.csv
 lda.state                   pyLDAvis_23.html  'Topic coherence plot.png'
 pending_images/             pyLDAvis_24.html


In [11]:
endpoint_url = "https://query.wikidata.org/sparql"

query_noodle = """SELECT ?noodle ?noodleLabel ?country_of_origin ?country_of_originLabel ?instance_of ?instance_ofLabel ?subclass_of ?subclass_ofLabel  ?Commons_category 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "   [AUTO_LANGUAGE],en". }
  ?noodle wdt:P279 wd:Q192874.
  OPTIONAL { ?noodle wdt:P495 ?country_of_origin. }
  OPTIONAL { ?noodle wdt:P31 ?instance_of. }
  OPTIONAL { ?noodle wdt:P279 ?subclass_of. }
  OPTIONAL { ?noodle wdt:P373 ?Commons_category. }  
}"""


df_noodle = get_results(endpoint_url, query_noodle)
df_noodle.head()

,noodle,noodleLabel,country_of_origin,country_of_originLabel,instance_of,instance_ofLabel,subclass_of,subclass_ofLabel,Commons_category
0,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
1,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q1142,Alsace,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
2,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
3,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q1142,Alsace,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
4,http://www.wikidata.org/entity/Q34156,Korean noodles,http://www.wikidata.org/entity/Q18097,Korea,None,None,http://www.wikidata.org/entity/Q192874,noodle,Noodles from Korea


In [12]:
noodle_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                         for item in df_noodle.groupby(\
                                                     ['noodleLabel'])['country_of_originLabel']\
                    .agg('count').index.values.tolist()])

noodle_list = sorted([re.sub("\-", " ",item) for item in noodle_list])
noodle_counter = Counter(noodle_list)

noodle_list = []
for key in noodle_counter.keys():
  if len(key) != 0:
    noodle_list.append(key)

print(len(noodle_list), noodle_list)

44 ['bakmi', 'black noodles', 'boat noodles', 'buckwheat noodles', 'buldak bokkeum myun', 'bánh hỏi', 'cart noodle', 'cellophane noodles', 'char kway teow', 'chinese noodles', 'chinkiang pot cover noodles', 'cold noodle', 'curry noodle', 'dragon beard noodles', 'extruded noodle', 'fried noodles', 'hokkien mee', 'hot dry noodles', 'jajangmyeon', 'japanese noodles', 'kadaif noodles', 'kesme', 'kishimen', 'knife cut noodle', 'korean noodles', 'laghman', 'mi rebus', 'mie ayam', 'mì', 'narrow lapsha', 'pancit', 'phat si io', 'ramen', 'ribbon noodle', 'rice noodles', 'singapore chow mein', 'soba', 'spätzle', 'sōmen', 'udon', 'vietnamese noodles', 'wide lapsha', 'wonton noodles', 'zhajiangmian']


In [17]:
endpoint_url = "https://query.wikidata.org/sparql"
query_soup = """SELECT ?subclass ?subclassLabel ?TasteAtlas_ID ?instance_of ?instance_ofLabel ?country_of_origin ?country_of_originLabel ?topic_s_main_template ?topic_s_main_templateLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?subclass wdt:P279 wd:Q41415.
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  OPTIONAL { ?subclass wdt:P495 ?country_of_origin. }
  OPTIONAL { ?subclass wdt:P1424 ?topic_s_main_template. }
}"""

df_soup = get_results(endpoint_url, query_soup)
df_soup.head()

,subclass,subclassLabel,TasteAtlas_ID,instance_of,instance_ofLabel,country_of_origin,country_of_originLabel,topic_s_main_template,topic_s_main_templateLabel
0,http://www.wikidata.org/entity/Q5183356,Cream Stew,None,http://www.wikidata.org/entity/Q4167410,Wikimedia disambiguation page,http://www.wikidata.org/entity/Q17,Japan,None,None
1,http://www.wikidata.org/entity/Q5188689,crossing the bridge noodles,over-bridge-rice-noodles,None,None,None,None,None,None
2,http://www.wikidata.org/entity/Q5191461,Crème Ninon,None,None,None,None,None,None,None
3,http://www.wikidata.org/entity/Q5194921,Fabes con almejas,None,None,None,None,None,None,None
4,http://www.wikidata.org/entity/Q5202905,Q5202905,None,None,None,None,None,None,None


In [21]:
endpoint_url = "https://query.wikidata.org/sparql"
query_soup = """SELECT ?item ?itemLabel WHERE {
  ?item wdt:P279 wd:Q41415.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

df_soup = get_results(endpoint_url, query_soup)
df_soup.head()
df_soup.shape

(402, 2)

In [27]:
df_soup.groupby('itemLabel')['item'].agg('count').index.values.tolist()

['15 bean soup',
 '2007 Vietnam food scare',
 'Acquacotta',
 'Aguadito de pollo',
 'Ajoblanco',
 'Alicot',
 'Amiedi',
 'Amish preaching soup',
 'Arabaşı soup',
 'Ash Reshteh',
 'Ashe doogh',
 'Asian soup',
 'Aush',
 'Bacon soup',
 'Bagnun',
 'Banga',
 'Batchoy',
 'Beyran',
 'Binignit',
 'Black sesame soup',
 'Bob chorba',
 'Bogeo',
 'Bookbinder Soup',
 'Borş de burechiuşe',
 'Bouillon',
 'Bouillon cube',
 'Bouneschlupp',
 'Brown Windsor soup',
 'Buddha Jumps Over the Wall',
 'Bun rieu',
 'Buridda',
 'Bún mắm',
 'Bún ốc',
 'Caldillo de perro',
 'Caldo de costilla',
 'Caldo de pollo',
 'Caldo de siete mares',
 'Caldo gallego',
 'Caldo tlalpeño',
 'Caldo verde',
 'Canja de Goa',
 'Cantonese seafood soup',
 'Caparrones',
 'Cazuela',
 'Celimpungan',
 'Chakna',
 'Changua',
 'Chankonabe',
 'Chicken and dumplings',
 'Chikhirtma',
 'Ching bo leung',
 'Chorba',
 'Chupe',
 'Chè bà ba',
 'Ciorbă',
 'Ciorbă de perișoare',
 'Clam soup',
 'Cocido lebaniego',
 'Cocido madrileño',
 'Cockchafer soup',
 

In [28]:
soup_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                         for item in df_soup.groupby(\
                                                     ['itemLabel'])['item'].agg('count').index.values.tolist()])

soup_list = sorted([re.sub("\-", " ",item) for item in soup_list])
soup_counter = Counter(soup_list)

soup_list = []
for key in soup_counter.keys():
  if len(key) != 0:
    soup_list.append(key)
print(len(soup_list),soup_list)

363 ['15 bean soup', '2007 vietnam food scare', 'acquacotta', 'aguadito de pollo', 'ajoblanco', 'alicot', 'amiedi', 'amish preaching soup', 'arabaşı soup', 'ash reshteh', 'ashe doogh', 'asian soup', 'aush', 'bacon soup', 'baeksuk', 'bagnun', 'banga', 'batchoy', 'bean soup', 'beer soup', 'beetroot soup', 'beyran', 'binignit', "bird's nest soup", 'bisque', 'black sesame soup', 'black soup', 'blood soup', 'bob chorba', 'bogeo', 'bookbinder soup', 'borş de burechiuşe', 'borș', 'bosintang', 'bouillon', 'bouillon cube', 'bouneschlupp', 'brown windsor soup', 'budae jjigae', 'buddha jumps over the wall', 'bun rieu', 'buridda', 'bún bò huế', 'bún mắm', 'bún ốc', 'cabbage soup', 'caldillo de perro', 'caldo de costilla', 'caldo de pollo', 'caldo de siete mares', 'caldo gallego', 'caldo tlalpeño', 'caldo verde', 'canja de goa', 'cantonese seafood soup', 'caparrones', 'cazuela', 'celimpungan', 'chakna', 'changua', 'chankonabe', 'cheese soup', 'chicken and dumplings', 'chicken soup', 'chikhirtma', '

In [34]:
endpoint_url = "https://query.wikidata.org/sparql"
query_rice_dish = """SELECT ?item ?itemLabel WHERE {
  ?item wdt:P279 wd:Q21976260.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""


"""SELECT ?is_a_list_of ?instance_of ?instance_ofLabel ?subclass_of ?subclass_ofLabel ?country_of_origin ?country_of_originLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?is_a_list_of wdt:P279 wd:Q21976260.
  OPTIONAL {  }
  OPTIONAL { ?is_a_list_of wdt:P5456 ?instance_of. }
  OPTIONAL { ?is_a_list_of wdt:P279 ?subclass_of. }
  OPTIONAL { ?is_a_list_of wdt:P495 ?country_of_origin. }
}
"""

df_rice_dish = get_results(endpoint_url,query_rice_dish)
print(df_rice_dish.shape)
df_rice_dish.head()


(122, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/Q20747991,Arroz poblano
1,http://www.wikidata.org/entity/Q20751180,Arroz a la tumbada
2,http://www.wikidata.org/entity/Q24945409,Hainanese curry rice
3,http://www.wikidata.org/entity/Q25700572,Arroz caldo
4,http://www.wikidata.org/entity/Q28155445,Albaloo polo


In [36]:

rice_dish_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                         for item in df_rice_dish.groupby(\
                                                          ['itemLabel'])['item'].agg('count').index.values.tolist()])

rice_dish_list = sorted([re.sub("\-", " ",item) for item in rice_dish_list])
rice_dish_counter = Counter(rice_dish_list)

rice_dish_list = []
for key in rice_dish_counter.keys():
  if len(key) != 0:
    rice_dish_list.append(key)
print(len(rice_dish_list),rice_dish_list)

113 ['agemochi', 'akki rotti', 'albaloo polo', 'ambur biryani', 'arroz a la cubana', 'arroz a la tumbada', 'arroz al horno', 'arroz caldo', 'arroz con pollo', 'arroz poblano', 'baji', 'banana leaf rice', 'beary biryani', 'beef biryani', 'bhatkali biryani', 'bibimbap', 'biryani', 'botamochi', 'cabidela', 'calas', 'chazuke', 'chicken biryani', 'chūkadon', 'ci fan tuan', 'claypot chicken rice', 'coconut rice', 'congee', 'cooked rice', 'curry rice', 'cơm lam', 'daifuku', 'dal bhat', 'delhi biryani', 'dindigul biryani', 'donburi', 'egg biryani', 'fish biryani', 'fried rice', 'gaifan', 'gohan', 'gyūdon', 'hainanese chicken rice', 'hainanese curry rice', 'hayashi rice', 'hoe deopbap', 'hokkien fried rice', 'idli', 'kabsa', 'kagami mochi', 'kalyani biryani', 'kamameshi', 'katsudon', 'ketupat', 'khao phat amerikan', 'kheer', 'kiribath', 'kuzhi paniyaram', 'lemang', 'lo mai gai', 'loco moco', 'malabar biryani', 'mansaf', 'maqluba', 'memoni biryani', 'mixed rice dish', 'morisqueta', 'moros y cris

In [41]:
endpoint_url = "https://query.wikidata.org/sparql"
query_salad = """SELECT ?item ?itemLabel WHERE {
  ?item wdt:P279 wd:Q9266.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

"""
SELECT ?subclass ?subclassLabel ?TasteAtlas_ID ?instance_of ?instance_ofLabel ?country_of_origin ?country_of_originLabel ?topic_s_main_template ?topic_s_main_templateLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?subclass wdt:P279 wd:Q9266.
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  OPTIONAL { ?subclass wdt:P495 ?country_of_origin. }
  OPTIONAL { ?subclass wdt:P1424 ?topic_s_main_template. }
}
"""

df_salad = get_results(endpoint_url,query_salad)
print(df_salad.shape)
df_salad.head()



(172, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/Q152834,fruit salad
1,http://www.wikidata.org/entity/Q152964,potato salad
2,http://www.wikidata.org/entity/Q177564,pico de gallo
3,http://www.wikidata.org/entity/Q185457,Q185457
4,http://www.wikidata.org/entity/Q209304,Gado-gado


In [42]:
salad_list = sorted([re.sub("q[0-9]+","",item.lower()) for item in df_salad.groupby(\
                                                          ['itemLabel'])['item'].agg('count').index.values.tolist()])
salad_counter = Counter(salad_list)

salad_list = []
for key in salad_counter.keys():
  if len(key) != 0:
    salad_list.append(key)
print(len(salad_list),salad_list)

140 ['a thoke', 'acar', 'ahtapot salata', 'arab salad', 'asinan', 'baba ghanoush', 'bean salad', 'beetroot salad', 'beyin salata', 'broccoli slaw', 'buljol', 'caesar salad', 'caprese salad', 'carrot salad', 'celery root salad', 'celery victor', 'cheese slaw', 'chef salad', 'chicken salad', 'chilean salad', 'chinese chicken salad', 'cobb salad', 'coleslaw', 'composed salad', 'cookie salad', 'crab louie', 'curtido', 'dakos', 'dessert salad', 'dressed herring', 'egg salad', 'eggplant salads and appetizers', 'egg–anchovy salad', 'eruca vesicaria', 'escalivada', 'esgarrat', 'esqueixada', 'fattoush', 'fiambre', 'fruit salad', 'gado-gado', 'garden salad', 'gavurdağı salatası', 'glass noodle salad', 'glasswort salad', 'glorified rice', 'golbaengi-muchim', 'goma-ae', 'greek salad', 'ham salad', 'herring salad', 'hmiss', 'israeli eggplant salad', 'israeli salad', 'jello salad', 'karedok', 'kelan antep', 'kinilnat', 'korean carrot salad', 'kosambari', 'kuluban', 'kurkkusalaatti', 'kısır', 'lalab'

In [47]:
endpoint_url = "https://query.wikidata.org/sparql"
query_dish = """SELECT ?item ?itemLabel WHERE {
  ?item wdt:P279 wd:Q746549.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

"""
SELECT ?subclass ?TasteAtlas_ID ?subclass_of ?subclass_ofLabel ?instance_of ?instance_ofLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P279 ?subclass_of. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  
  ?subclass wdt:P279 wd:Q746549.
}
"""


df_dish = get_results(endpoint_url,query_dish)
print(df_dish.shape)
df_dish.head()

(906, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/Q177,pizza
1,http://www.wikidata.org/entity/Q208,rakfisk
2,http://www.wikidata.org/entity/Q375,waffle
3,http://www.wikidata.org/entity/Q738,croque-monsieur
4,http://www.wikidata.org/entity/Q6493,Labskaus


In [48]:
dish_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                    for item in df_dish.groupby(\
                                                ['itemLabel'])['item'].agg('count').index.values.tolist()])

dish_list = sorted([re.sub("\-", " ",item) for item in dish_list])

dish_counter = Counter(dish_list)

dish_list = []
for key in dish_counter.keys():
  if len(key) != 0:
    dish_list.append(key)
print(len(dish_list),dish_list)

605 ['a gei', 'achar', 'acquasale', 'agedashi dofu', 'ajapsandali', 'akuri', 'alheira', 'aloo gobi', 'ambelopoulia', 'ambrosia and nectar', 'ambuyat', 'ampo snack', 'anticuchos', 'apohtin', 'apple sauce', 'arabaşı soup', 'aranese cream', 'arroz al horno', 'arroz gordo', 'arroz junto', 'aspic', 'aushak', 'avocado toast', 'ayam pansuh', 'azu', 'azumadon', 'bacalhau à brás', 'bacon, egg and cheese sandwich', 'badrijani', 'bagel and cream cheese', 'baingan bartha', 'bajjaria', 'baked beans', 'baked potato', 'balichão', 'balut', 'balık ekmek', 'banana bread', 'banana dish', 'banbury cake', 'banku', 'bannock', 'banusz', 'barley based dishes', 'basbousa', 'beef bourguignon', 'beef brisket', 'beef chow fun', 'beef noodle soup', 'beef stroganoff', 'beef wellington', 'beignet', 'belgian endive with ham', 'bethmännchen', 'beutelwurst', 'bienmesabe', 'binyang dog leg dish', 'biryani', 'bison burgers', 'bissara', 'black garlic', 'black peas', 'black sesame soup', 'blood dishes', 'blunkett salad', '

In [52]:
endpoint_url = "https://query.wikidata.org/sparql"
query_drink = """SELECT ?item ?itemLabel WHERE {
  ?item wdt:P279 wd:Q40050.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

"""SELECT ?subclass ?TasteAtlas_ID ?subclass_of ?subclass_ofLabel ?instance_of ?instance_ofLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P279 ?subclass_of. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  ?subclass wdt:P279 wd:Q40050.
}"""

df_drink = get_results(endpoint_url,query_drink)
print(df_drink.shape)
df_drink.head()

(311, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/Q154,alcoholic beverage
1,http://www.wikidata.org/entity/Q7892,drinking water
2,http://www.wikidata.org/entity/Q8486,coffee
3,http://www.wikidata.org/entity/Q8492,juice
4,http://www.wikidata.org/entity/Q8495,milk


In [53]:
drink_list = sorted([re.sub("q[0-9]+","",item.lower())\
                     for item in df_drink.groupby(\
                                                  ['itemLabel'])['item'].agg('count').index.values.tolist()])

drink_list = sorted([re.sub("\-", " ",item) for item in drink_list])

drink_counter = Counter(drink_list)

drink_list = []
for key in drink_counter.keys():
  if len(key) != 0:
    drink_list.append(key)
print(len(drink_list),drink_list)

281 ['1519 tequila', '3 a.m. vodka', '4 copas', '5 hour energy', 'accelerade', 'affogato', 'aguapanela', 'air sirap', 'alazani', 'alcoholic beverage', 'ale 8 one', 'all sport', 'amazake', 'anestasia vodka', 'apéritif', 'arctic velvet', 'arwa', 'avion tequila', 'ayran', 'barbasol', 'beef tea', 'beep', 'belaya rus vodka', "benjamin prichard's tennessee whiskey", 'biała dama', 'black cow vodka', 'blue', 'blue moon', 'boilo', 'boj', 'bondi blonde', 'bong spirit vodka', 'boodles british gin', 'bovonto', 'boyd & blair', 'boza', 'brexitovka', 'brisbane bitter', 'buttermilk koldskål', 'caffeinated drink', 'caffè americano', 'caffè corretto', 'café rica', 'cape north vodka', 'carbonated beverage', 'casa dragones', 'casa noble', 'castello mio sambuca', 'caudle', 'cel ray', 'chal', 'chernihivske', 'chicago root beer', 'chicha de jora', 'chicha morada', 'choc ola', 'chuflay', 'cirrus vodka', 'citadelle', 'citron pressé', 'citronnade', 'clique', 'clix malt liquor', 'coca wine', 'cocaine', 'cochlear

In [57]:
endpoint_url = "https://query.wikidata.org/sparql"
query_hot_bev = """SELECT ?item ?itemLabel WHERE {
  ?item wdt:P279 wd:Q19359564.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

"""SELECT ?subclass ?TasteAtlas_ID ?subclass_of ?subclass_ofLabel ?instance_of ?instance_ofLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P279 ?subclass_of. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  ?subclass wdt:P279 wd:Q19359564.
}"""

df_hot_bev = get_results(endpoint_url,query_hot_bev)
print(df_hot_bev.shape)
df_hot_bev.head()

(13, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/Q6097,tea
1,http://www.wikidata.org/entity/Q13261,hot chocolate
2,http://www.wikidata.org/entity/Q59072,cortado
3,http://www.wikidata.org/entity/Q159774,cappuccino
4,http://www.wikidata.org/entity/Q379932,herbal tea


In [58]:
hot_bev_list = sorted([re.sub("q[0-9]+","",item.lower())\
                       for item in df_hot_bev.groupby(\
                                                    ['itemLabel'])['item'].agg('count').index.values.tolist()])
hot_bev_list = sorted([re.sub("\-", " ",item) for item in hot_bev_list])

hot_bev_counter = Counter(hot_bev_list)

hot_bev_list = []
for key in hot_bev_counter.keys():
  if len(key) != 0:
    hot_bev_list.append(key)
print(len(hot_bev_list),hot_bev_list)

11 ['cafe mocha', 'cappuccino', 'cortado', 'herbal tea', 'hot chocolate', 'hot citron', 'irish coffee', 'italian coffee', 'kurdish coffee', 'tea', 'warm alcoholic drink']


In [62]:
endpoint_url = "https://query.wikidata.org/sparql"
query_coffee_drink = """SELECT ?item ?itemLabel WHERE {
  ?item wdt:P279 wd:Q37756327.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

"""SELECT ?subclass ?TasteAtlas_ID ?subclass_of ?subclass_ofLabel ?instance_of ?instance_ofLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P279 ?subclass_of. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  ?subclass wdt:P279 wd:Q37756327.
}"""

df_coffee_drink = get_results(endpoint_url,query_coffee_drink)
print(df_coffee_drink.shape)
df_coffee_drink.head()

(50, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/Q8486,coffee
1,http://www.wikidata.org/entity/Q20674,Pharisees
2,http://www.wikidata.org/entity/Q22929,latte macchiato
3,http://www.wikidata.org/entity/Q59072,cortado
4,http://www.wikidata.org/entity/Q62449,flat white


In [63]:
coffee_drink_list = sorted([re.sub("q[0-9]+","",item.lower())\
                       for item in df_coffee_drink.groupby(\
                                                    ['itemLabel'])['item'].agg('count').index.values.tolist()])
coffee_drink_list = sorted([re.sub("\-", " ",item) for item in coffee_drink_list])

coffee_drink_counter = Counter(coffee_drink_list)

coffee_drink_list = []
for key in coffee_drink_counter.keys():
  if len(key) != 0:
    coffee_drink_list.append(key)
print(len(coffee_drink_list),coffee_drink_list)

46 ['algerian coffee of prague', 'asiático', 'bicerin', 'black russian', 'cafe mocha', 'caffè', 'caffè con panna', 'caffè crema', 'caffè macchiato', 'café con leche', 'café de olla', 'café touba', 'cappuccino', 'coffee', 'cold brew coffee', 'cortado', 'cuban espresso', 'egg coffee', 'einspänner', 'espressino', 'flat white', 'frappé coffee', 'iced coffee', 'indian filter coffee', 'instant coffee', 'ipoh white coffee', 'italian coffee', 'kaffepunch', 'karsk', 'kopi o', 'kopi tubruk', 'latte', 'latte macchiato', 'liqueur coffee', 'long black', 'marocchino', 'mazagran', 'milk coffee', 'mustang coffee', 'mırra', 'nitro cold brew coffee', 'pharisees', 'pumpkin spice latte', 'red eye', 'vietnamese iced coffee', 'yuanyang']


In [66]:
endpoint_url = "https://query.wikidata.org/sparql"
query_taste_atlas = """SELECT ?subclass ?subclassLabel ?TasteAtlas_ID WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
}
"""
df_taste_atlas = get_results(endpoint_url,query_taste_atlas)
print(df_taste_atlas.shape)
df_taste_atlas.head()


(6079, 3)


,subclass,subclassLabel,TasteAtlas_ID
0,http://www.wikidata.org/entity/Q178,pasta,pastasicily
1,http://www.wikidata.org/entity/Q177,pizza,pizza
2,http://www.wikidata.org/entity/Q616,Beaujolais wine,beaujolais
3,http://www.wikidata.org/entity/Q732,Francesinha,francesinha
4,http://www.wikidata.org/entity/Q7633290,sudan,sudan


In [68]:
taste_atlas_list = sorted([re.sub("q[0-9]+","",item.lower())\
                           for item in df_taste_atlas.groupby(\
                                                              ['subclassLabel'])['subclass'].agg('count').index.values.tolist()])

taste_atlas_list = sorted([re.sub("\-", " ",item) for item in taste_atlas_list])

taste_atlas_counter = Counter(taste_atlas_list)

taste_atlas_list = []
for key in taste_atlas_counter.keys():
  if len(key) != 0:
    taste_atlas_list.append(key)
print(len(taste_atlas_list),taste_atlas_list)

5055 ["'mpanatigghi", "'nduja", "'ota 'ika", '.amaro ramazzotti', '20th century', '7 and 7', 'a gei', 'aachener printen', 'aam panna', 'aamras', 'abbaye de citeaux', 'abbaye de tamié', 'abertam cheese', 'abgoosht', 'abondance', 'aborrajado', 'abricotine', 'absinthe', 'aburaage', 'acar', 'acarajé', 'acceglio (cheese)', 'acini di pepe', 'ackee and saltfish', 'acqua pazza', 'acquacotta', 'adana kebabı', 'adjika', 'adobo', 'afelia', 'affogato', 'afghan biscuit', 'afritada', 'afternoon snack', "afuega'l pitu", 'agedashi dofu', 'agent orange', 'aggala', 'aglianico', 'agliata', 'agneau du périgord', 'agnolotti', 'agrodolce', 'agua de sevilla', 'agua de valencia', 'aguachile', 'aguadito de pollo', 'aguas frescas', 'agwijjim', 'ahlgrens bilar', 'aioli', 'aish merahrah', 'ajapsandali', 'ajdovi žganci', 'aji de gallina', 'ajiaco', 'ajilimójili', 'ajoblanco', 'ajvar', 'akafuku', 'akanés', 'akkawi', 'akki rotti', 'akple', 'akumaki', 'akvavit', 'al pastor', 'alabama slammer', 'alambre', 'albariño', 

In [71]:
full_list = sorted(dish_list + rice_dish_list + soup_list + noodle_list + salad_list + drink_list + hot_bev_list + coffee_drink_list + taste_atlas_list)
full_list = sorted([item for item in full_list if not item[0].isdigit()])

full_counter = Counter(full_list)
full_list=[]
for key in full_counter.keys():
  if len(key) != 0:
    full_list.append(key)
print(len(full_list))

5997


In [69]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=ada8dc327d5132192f73efffd7fe686734f5d4a2a838af4935fe5107795acfe5
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import wikipedia
import time

In [72]:
major_list = []

for item in full_list:
  pattern_list = []
  pattern_dict = {}
  for word in item.split():
    token_dict = {}
    token_dict['LOWER'] = word
    pattern_list.append(token_dict)

  pattern_dict['label'] = 'dish'
  pattern_dict['pattern'] = pattern_list

  major_list.append(pattern_dict)

major_list


[{'label': 'dish', 'pattern': [{'LOWER': "'mpanatigghi"}]},
 {'label': 'dish', 'pattern': [{'LOWER': "'nduja"}]},
 {'label': 'dish', 'pattern': [{'LOWER': "'ota"}, {'LOWER': "'ika"}]},
 {'label': 'dish', 'pattern': [{'LOWER': '.amaro'}, {'LOWER': 'ramazzotti'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'a'}, {'LOWER': 'gei'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'a'}, {'LOWER': 'thoke'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'aachener'}, {'LOWER': 'printen'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'aam'}, {'LOWER': 'panna'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'aamras'}]},
 {'label': 'dish',
  'pattern': [{'LOWER': 'abbaye'}, {'LOWER': 'de'}, {'LOWER': 'citeaux'}]},
 {'label': 'dish',
  'pattern': [{'LOWER': 'abbaye'}, {'LOWER': 'de'}, {'LOWER': 'tamié'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'abertam'}, {'LOWER': 'cheese'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'abgoosht'}]},
 {'label': 'dish', 'pattern': [{'LOWER': 'abondance'}]},
 {'label': 'dish', '

In [0]:
dish_keyword_patterns = np.array(major_list)
np.savez("dish_keyword_patterns", dish_keyword_patterns)

In [0]:

def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))


def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [75]:
for item in major_list:
  dump_jsonl([item],'patterns.jsonl', append=True)

Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 records to patterns.jsonl
Wrote 1 record

In [76]:
wiki_info_list = []
items_with_issues_list = []
for item in taste_atlas_list:
  time.sleep(1)
  wiki_dict = {}
  try:
    wiki_dict['name'] = item
    wiki_dict['page'] = wikipedia.page(item).url
    wiki_dict['summary'] = wikipedia.summary(item)
    wiki_dict['image_ref'] = wikipedia.page(item).images[0]
  except:
    items_with_issues_list.append(item)
  wiki_info_list.append(wiki_dict)


    

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [77]:
for item in wiki_info_list:
  dump_jsonl([item],'taste_atlas_info.jsonl', append=True)

Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_info.jsonl
Wrote 1 records to taste_atlas_inf

In [78]:
items_with_issues_list

['a gei',
 'acqua pazza',
 'alcamo wine',
 'alu tama',
 'amba',
 'americano',
 'angel face',
 'anju',
 'apohtin',
 'ardi gasna',
 'arjona tuberosa',
 'açma',
 'baba',
 'baci di dama',
 'banket',
 'bansh',
 'bantan',
 'bavarian beers',
 'bavette',
 'bazeen',
 'bear claw',
 'beaufort',
 'bergamottes de nancy',
 "bey's soup",
 'beyran',
 'biscotti del lagaccio',
 'bitter',
 'bleu de termignon',
 'bloody mary',
 'boil up',
 'boodog',
 'bosna',
 'boterkoek',
 'bouchee a la reine',
 'bouchon de sancerre',
 'bourride',
 'brandy sour',
 'brass monkey',
 'briquette de brebis',
 'brut cocktail',
 'budu',
 'budyń',
 'bukayo',
 'burgossan cheese',
 'buttery',
 'bœuf de bazas igp',
 'cable car',
 'cadi',
 'calas',
 'caprice des dieux',
 'carac',
 'caravane',
 'carcerato',
 'carne de morucha de salamanca',
 'carne de vacuno del país vasco',
 'carp soup',
 'causa a la limeña',
 'cava',
 'cayeye',
 'chacha',
 'chairo',
 'cham cham',
 'chambolle musigny wine',
 'chambord',
 'champ',
 'champús',
 'chape

In [79]:
len(items_with_issues_list)

339

In [0]:
print(wikipedia.summary(taste_atlas_list[6]))
print(wikipedia.page(taste_atlas_list[6]).url)
print(wikipedia.page(taste_atlas_list[6]).content)


Aamras (or amras) is a sweet dish featuring in the cuisine of the Indian subcontinent and made from the pulp of the mango fruit. The pulp of a ripe mango is extracted, usually by hand, and is consumed together with pooris or chapati(Indian breads). Sometimes ghee and milk are added to the pulp to enhance its flavour. Sugar is also added to adjust the sweetness.It is often had at celebrations and weddings with cardamon and chopped fruits.
A regional version of aamras is a popular dessert in Rajasthani cuisine and Marwari, Marathi, and  Gujarati homes, especially during festivities.
Since the fruit is seasonal, being harvested at the end of summer, the need to preserve the fruit in the form of pulp has given rise to a moderately large mango-processing industry.
https://en.wikipedia.org/wiki/Aamras
Aamras (or amras) is a sweet dish featuring in the cuisine of the Indian subcontinent and made from the pulp of the mango fruit. The pulp of a ripe mango is extracted, usually by hand, and is c

In [80]:
df_taste_atlas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6079 entries, 0 to 6078
Data columns (total 3 columns):
subclass         6079 non-null object
subclassLabel    6079 non-null object
TasteAtlas_ID    6079 non-null object
dtypes: object(3)
memory usage: 142.6+ KB


In [82]:
df_taste_atlas.query('subclassLabel == "Bosna"')

,subclass,subclassLabel,TasteAtlas_ID
3323,http://www.wikidata.org/entity/Q358507,Bosna,bosna


In [83]:
wikipedia.set_lang("de")
wiki_info_list_de = []
items_with_issues_list_de = []
for item in items_with_issues_list:
  time.sleep(1)
  wiki_dict_de = {}
  try:
    wiki_dict_de['name'] = item
    wiki_dict_de['page'] = wikipedia.page(item).url
    wiki_dict_de['summary'] = wikipedia.summary(item)
    wiki_dict_de['image_ref'] = wikipedia.page(item).images[0]
  except:
    items_with_issues_list_de.append(item)
  wiki_info_list_de.append(wiki_dict_de)

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [84]:
for item in wiki_info_list_de:
  dump_jsonl([item],'taste_atlas_info_de.jsonl', append=True)

Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 records to taste_atlas_info_de.jsonl
Wrote 1 re

In [85]:
len(items_with_issues_list_de)

188

In [86]:
wikipedia.set_lang("it")
wiki_info_list_it = []
items_with_issues_list_it = []
for item in items_with_issues_list_de:
  time.sleep(1)
  wiki_dict_it = {}
  try:
    wiki_dict_it['name'] = item
    wiki_dict_it['page'] = wikipedia.page(item).url
    wiki_dict_it['summary'] = wikipedia.summary(item)
    wiki_dict_it['image_ref'] = wikipedia.page(item).images[0]
  except:
    items_with_issues_list_it.append(item)
  wiki_info_list_it.append(wiki_dict_it)

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [87]:
for item in wiki_info_list_it:
  dump_jsonl([item],'taste_atlas_info_it.jsonl', append=True)

Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 records to taste_atlas_info_it.jsonl
Wrote 1 re

In [88]:
len(items_with_issues_list_it)

117

In [92]:
wikipedia.set_lang("es")
wiki_info_list_es = []
items_with_issues_list_es = []
for item in items_with_issues_list_it:
  time.sleep(1)
  wiki_dict_es = {}
  try:
    wiki_dict_es['name'] = item
    wiki_dict_es['page'] = wikipedia.page(item).url
    wiki_dict_es['summary'] = wikipedia.summary(item)
    wiki_dict_es['image_ref'] = wikipedia.page(item).images[0]
  except:
    items_with_issues_list_es.append(item)
  wiki_info_list_es.append(wiki_dict_es)

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [93]:
for item in wiki_info_list_es:
  dump_jsonl([item],'taste_atlas_info_es.jsonl', append=True)

Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 records to taste_atlas_info_es.jsonl
Wrote 1 re

In [94]:
len(items_with_issues_list_es)

77

In [96]:
wikipedia.set_lang("fr")
wiki_info_list_fr = []
items_with_issues_list_fr = []
for item in items_with_issues_list_es:
  time.sleep(1)
  wiki_dict_fr = {}
  try:
    wiki_dict_fr['name'] = item
    wiki_dict_fr['page'] = wikipedia.page(item).url
    wiki_dict_fr['summary'] = wikipedia.summary(item)
    wiki_dict_fr['image_ref'] = wikipedia.page(item).images[0]
  except:
    items_with_issues_list_fr.append(item)
  wiki_info_list_fr.append(wiki_dict_fr)

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [97]:
for item in wiki_info_list_fr:
  dump_jsonl([item],'taste_atlas_info_fr.jsonl', append=True)

Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 records to taste_atlas_info_fr.jsonl
Wrote 1 re

In [98]:
len(items_with_issues_list_fr)

51

In [0]:
wikipedia.set_lang("fy")
wiki_info_list_fy = []
items_with_issues_list_fy = []
for item in items_with_issues_list_fr:
  time.sleep(1)
  wiki_dict_fy = {}
  try:
    wiki_dict_fy['name'] = item
    wiki_dict_fy['page'] = wikipedia.page(item).url
    wiki_dict_fy['summary'] = wikipedia.summary(item)
    wiki_dict_fy['image_ref'] = wikipedia.page(item).images[0]
  except:
    items_with_issues_list_fy.append(item)
  wiki_info_list_fy.append(wiki_dict_fy)

In [101]:
for item in wiki_info_list_fy:
  dump_jsonl([item],'taste_atlas_info_fy.jsonl', append=True)

Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 records to taste_atlas_info_fy.jsonl
Wrote 1 re

In [103]:
wiki_info_list_fy

[{'name': 'alcamo wine'},
 {'name': 'bavarian beers'},
 {'name': "bey's soup"},
 {'name': 'beyran'},
 {'name': 'boil up'},
 {'name': 'boterkoek'},
 {'name': 'budyń'},
 {'name': 'burgossan cheese'},
 {'name': 'carp soup'},
 {'name': 'chacha'},
 {'name': "cheese from l'alt urgell y la cerdanya"},
 {'name': 'chistorramesta'},
 {'name': 'cokodok'},
 {'name': 'fried srhimp'},
 {'name': 'gala',
  'page': 'https://fy.wikipedia.org/wiki/Frysk_Sjongers_Gala',
  'summary': "It Frysk Sjongers Gala is in muzykfeest dêr't it Fryske liet sintraal stiet. Dit muzykfestival wurdt oan de ein fan it jier holden yn De Lawei yn Drachten. It Gala, dat sûnt 2012 bestiet, wie in inisjatyf fan Piter Wilkens, Griet Wiersma, Anneke Douma en Gurbe Douwstra. It orkest bestiet út Fryske muzikanten, mei in eftergrûnkoar, in strykkwartet en in blazerssseksje. De muzikale lieding is yn hannen fan Peter van der Zwaag. Op it programma steane neist nije lieten ek medleys fan lieten út de ôfrûne desennia. It is in griemma

In [102]:
len(items_with_issues_list_fy)

50

In [106]:
df_json = pd.read_json('taste_atlas_info.jsonl', lines=True)
df_json

,name,page,summary,image_ref
0,'mpanatigghi,https://en.wikipedia.org/wiki/Empanada,An empanada is a type of baked or fried turnov...,https://upload.wikimedia.org/wikipedia/commons...
1,'nduja,https://en.wikipedia.org/wiki/%27Nduja,'Nduja (Calabrian: [nˈduːja]) is a particularl...,https://upload.wikimedia.org/wikipedia/commons...
2,'ota 'ika,https://en.wikipedia.org/wiki/%27Ota_%27ika,"'Ota ika is a Polynesian dish, similar to Lati...",https://upload.wikimedia.org/wikipedia/commons...
3,.amaro ramazzotti,https://en.wikipedia.org/wiki/Amaro_(liqueur),"Amaro (Italian for ""bitter"") is an Italian her...",https://upload.wikimedia.org/wikipedia/commons...
4,20th century,https://en.wikipedia.org/wiki/20th_Century_Fox,Twentieth Century Fox Film Corporation (colloq...,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...
5050,štramberk ears,https://en.wikipedia.org/wiki/%C5%A0tramberk_ears,Štramberk ears (Czech: Štramberské uši) is a M...,https://upload.wikimedia.org/wikipedia/commons...
5051,štruklji,https://en.wikipedia.org/wiki/%C5%A0truklji,"Štruklji are a traditional Slovene dish, compo...",https://upload.wikimedia.org/wikipedia/commons...
5052,żymlok,https://en.wikipedia.org/wiki/Kaszanka,Kaszanka is a traditional blood sausage in eas...,https://upload.wikimedia.org/wikipedia/commons...
5053,žemlovka,https://en.wikipedia.org/wiki/%C5%BDemlovka,Žemlovka (Moravian dialect: zemlbába) is a swe...,https://upload.wikimedia.org/wikipedia/commons...


In [0]:
df_json[['name','summary']].dropna().to_csv('test_summaries.csv')